# Data Cleaning 

#### 1. Import pandas library.

In [22]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data. 


In [23]:
#sql

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/dataset/Stats).

In [24]:
#sql

#### 4. Import the users table.

In [43]:
user=pd.read_csv("/Users/ramon/Desktop/Ironhack/June/Module 1/Week 2/2-Data cleaning/Datasets as a CSV/users.csv")
user = user.drop("Unnamed: 0", axis = 1)
user

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5
...,...,...,...,...,...
40320,55743,1,0,0,0
40321,55744,6,1,0,0
40322,55745,101,0,0,0
40323,55746,106,1,0,0


#### 5. Rename Id column to userId.

In [44]:
print(newcol.columns)
newcol=user.rename(columns={"userId":"ID"})
newcol

Index(['Unnamed: 0', 'ID', 'Reputation', 'Views', 'UpVotes', 'DownVotes'], dtype='object')


,ID,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5
...,...,...,...,...,...
40320,55743,1,0,0,0
40321,55744,6,1,0,0
40322,55745,101,0,0,0
40323,55746,106,1,0,0


#### 6. Import the posts table. 

In [48]:
posts=pd.read_csv("/Users/ramon/Desktop/Ironhack/June/Module 1/Week 2/2-Data cleaning/Datasets as a CSV/posts.csv")
posts=posts.drop("Unnamed: 0", axis = 1)
posts

,PostId,userId,Score,ViewCount,CommentCount
0,1,8.0,23,1278.0,1
1,2,24.0,22,8198.0,1
2,3,18.0,54,3613.0,4
3,4,23.0,13,5224.0,2
4,5,23.0,81,NaN,3
...,...,...,...,...,...
91971,115374,805.0,2,NaN,2
91972,115375,49365.0,0,9.0,0
91973,115376,55746.0,1,5.0,2
91974,115377,805.0,0,NaN,0


#### 7. Rename Id column to postId and OwnerUserId to userId.

In [49]:
posts.columns
newcol_p=posts.rename(columns={"PostId":"Idcolumns","userId":"OwnerUserid"})
newcol_p

,Idcolumns,OwnerUserid,Score,ViewCount,CommentCount
0,1,8.0,23,1278.0,1
1,2,24.0,22,8198.0,1
2,3,18.0,54,3613.0,4
3,4,23.0,13,5224.0,2
4,5,23.0,81,NaN,3
...,...,...,...,...,...
91971,115374,805.0,2,NaN,2
91972,115375,49365.0,0,9.0,0
91973,115376,55746.0,1,5.0,2
91974,115377,805.0,0,NaN,0


#### 8. Define new dataframes for users and posts with the following selected columns:
**users_sliced columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts_sliced columns**: postId, Score, userId, ViewCount, CommentCount

In [53]:
users_sliced_columns=newcol.rename(columns={"ID":"userId"})
posts_sliced_columns=newcol_p.rename(columns={"Idcolumns":"postId","OwnerUserid":"Score","Score":"userId"})
posts_sliced_columns

,postId,Score,userId,ViewCount,CommentCount
0,1,8.0,23,1278.0,1
1,2,24.0,22,8198.0,1
2,3,18.0,54,3613.0,4
3,4,23.0,13,5224.0,2
4,5,23.0,81,NaN,3
...,...,...,...,...,...
91971,115374,805.0,2,NaN,2
91972,115375,49365.0,0,9.0,0
91973,115376,55746.0,1,5.0,2
91974,115377,805.0,0,NaN,0


#### 9. Merge the two dataframes created in the step above (8), users_sliced and posts_sliced. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [77]:
posts_sliced_columns=posts_sliced_columns.merge(right=users_sliced_columns, how="inner")
posts_sliced_columns

,postId,Score,userId,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1,8.0,23,1278.0,1,308.0,52.0,34.0,1.0
1,10,24.0,23,21925.0,4,308.0,52.0,34.0,1.0
2,124,131.0,23,1131.0,0,308.0,52.0,34.0,1.0
3,131,5.0,23,NaN,0,308.0,52.0,34.0,1.0
4,282,81.0,23,NaN,3,308.0,52.0,34.0,1.0
...,...,...,...,...,...,...,...,...,...
46694,83136,37729.0,110,25744.0,23,443.0,59.0,9.0,1.0
46695,83162,37741.0,99,NaN,8,151.0,42.0,3.0,0.0
46696,86015,37418.0,99,29303.0,35,151.0,42.0,3.0,0.0
46697,83262,5836.0,135,NaN,8,101.0,1.0,2.0,0.0


#### 10. How many missing values do you have in your merged dataframe? On which columns?

In [78]:
missing_values=posts_sliced_columns.isnull().sum().sum()
print("there are",missing_values,"missing values")
posts_sliced_columns.info()

there are 27398 missing values
<class 'pandas.core.frame.DataFrame'>
Int64Index: 46699 entries, 0 to 46698
Data columns (total 9 columns):
postId          46699 non-null int64
Score           45937 non-null float64
userId          46699 non-null int64
ViewCount       20063 non-null float64
CommentCount    46699 non-null int64
Reputation      46699 non-null float64
Views           46699 non-null float64
UpVotes         46699 non-null float64
DownVotes       46699 non-null float64
dtypes: float64(6), int64(3)
memory usage: 3.6 MB


#### 11. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

In [84]:
posts_sliced_columns["ViewCount"]=posts_sliced_columns["ViewCount"].fillna(0)
posts_sliced_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46699 entries, 0 to 46698
Data columns (total 9 columns):
postId          46699 non-null int64
Score           45937 non-null float64
userId          46699 non-null int64
ViewCount       46699 non-null float64
CommentCount    46699 non-null int64
Reputation      46699 non-null float64
Views           46699 non-null float64
UpVotes         46699 non-null float64
DownVotes       46699 non-null float64
dtypes: float64(6), int64(3)
memory usage: 3.6 MB


#### 12. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [86]:
posts_sliced_columns.ViewCount=posts_sliced_columns.ViewCount.astype(int)
posts_sliced_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46699 entries, 0 to 46698
Data columns (total 9 columns):
postId          46699 non-null int64
Score           45937 non-null float64
userId          46699 non-null int64
ViewCount       46699 non-null int32
CommentCount    46699 non-null int64
Reputation      46699 non-null float64
Views           46699 non-null float64
UpVotes         46699 non-null float64
DownVotes       46699 non-null float64
dtypes: float64(5), int32(1), int64(3)
memory usage: 3.4 MB
